In [ ]:
%pip install selenium

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
from dotenv import load_dotenv
from selenium.webdriver.common.by import By
import re

urls=["http://www.stand1318.com/bbs/board.php?bo_table=5_2","https://www.facebook.com/forten1318?locale=ko_KR","https://www.facebook.com/dreams9570/"]

# Create a new instance of the Firefox driver
driver = webdriver.Chrome()


post_list=[]

for i in range(len(urls)):
    if "facebook" in urls[i]:
        # Load the environment variables from the .env file
        load_dotenv()
        email = os.getenv('EMAIL')
        password = os.getenv('PASSWORD')

        # Set up the Selenium webdriver
        driver.get('https://www.facebook.com')

        # Find the email and password fields, and enter your login credentials
        email_field = driver.find_element(By.ID,'email')
        email_field.send_keys(email)

        password_field = driver.find_element(By.ID,'pass')
        password_field.send_keys(password)

        # Find the login button and click it
        login_button = driver.find_element(By.NAME,'login')
        login_button.click()

        # Wait for the page to load and navigate to the target page
        time.sleep(5)
        driver.get(urls[i])


        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(5)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            posts = soup.find_all('div', {'class': 'x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z'})
            if len(posts) >= 5:
                break
        
        # Print the post text and url
        for post in posts:
            post_text = post.find('div', {'class': 'x11i5rnm xat24cr x1mh8g0r x1vvkbs xdj266r x126k92a'})
            post_url = post.find('a', {'class': 'x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g xt0b8zv xo1l8bm'})
            text = post_text.text.strip()
            first_sentence = re.split('[._\-\]]\s*',text)[0]
            if '[' in first_sentence:
                first_sentence = first_sentence + ']'

            if post_text is not None and post_url is not None:
                #print(first_sentence)
                #print(post_url['href'])
                post_item ={
                'title': first_sentence,
                'URL': post_url['href']
                }
                post_list.append(post_item)
                

    else:
        # Navigate to the website
        driver.get(urls[i])
        # Wait for the page to load
        driver.implicitly_wait(10)

        # Get the HTML source of the page
        html = driver.page_source

        # Parse the HTML with BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Find all the posts
        posts = soup.find_all('div', class_='gall_con')

        # Loop through each post and extract the title and date
        for post in posts:
            # Find the title and date elements
            title_elem = post.find('a', class_='bo_tit')
            date_elem = post.find('span', class_='gall_date')
            url_elem = post.find('a',class_='bo_tit')
            img_elem = post.find('img')

            
            # Extract the title and date text
            title = title_elem.text.strip()
            date_text = date_elem.text.strip()
            date_str = date_text.split(' ')[-1]


            post_item ={
                'title': title,
                'URL': url_elem['href'],
                'Img': img_elem['src']
            }
            post_list.append(post_item)


df = pd.DataFrame(post_list)
print(df)

# Close the browser
driver.quit()


In [7]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore


cred = credentials.Certificate("teenhelper-93c4e-firebase-adminsdk-293wr-2bd24cb784.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

data_dict = df.to_dict('list')
df_notification = df.drop(['URL'],axis=1,inplace=True)

def save(collection_id, document_id, data):
    db.collection(collection_id).document(str(document_id)).set(data)


for i,row in df.iterrows():
    save(
        collection_id="Notification", 
        document_id=i,
        data=row.to_dict()
    )




In [ ]:
from datetime import datetime
from pytz import timezone

tz = datetime.now(timezone('Asia/Seoul'))
time = tz.strftime('%Y-%m-%d')

df['timeStamp']=time



def save(collection_id, document_id, data):
    db.collection(collection_id).document(str(document_id)).set(data)


for i,row in df.iterrows():
    save(
        collection_id="Notification2", 
        document_id=i,
        data=row.to_dict()
    )